In [ ]:
import requests # Used for making HTTP requests to fetch data from web APIs.
import pandas as pd # Used for data manipulation and analysis, especially with tabular data (DataFrames).
from datetime import datetime # Used for working with dates and times, specifically to calculate time to expiration.
from math import log, sqrt, exp # Mathematical functions: natural logarithm, square root, and exponential.
from scipy.stats import norm # Scientific Python library for statistical functions, specifically the cumulative distribution function (CDF) of the normal distribution.
from scipy.optimize import brentq
import yfinance as yf # Import the yfinance library for fetching financial data.

# ----------------------------
# 1. Black-Scholes IV Function
# ----------------------------

def black_scholes_price(S, K, T, r, sigma, option_type="call"):
    """
    Calculates the theoretical price of a European option using the Black-Scholes model.

    Args:
        S (float): Current price of the underlying asset.
        K (float): Strike price of the option.
        T (float): Time to expiration in years (e.g., 0.5 for 6 months).
        r (float): Risk-free interest rate (annualized, e.g., 0.05 for 5%).
        sigma (float): Volatility of the underlying asset (annualized).
        option_type (str): Type of the option, either "call" or "put".

    Returns:
        float: The theoretical Black-Scholes option price.
    """
    # Calculate d1 parameter of the Black-Scholes formula.
    # d1 incorporates the asset price, strike price, time, risk-free rate, and volatility.
    d1 = (log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrt(T))
    # Calculate d2 parameter, which is derived from d1.
    d2 = d1 - sigma * sqrt(T)

    # Calculate option price based on option type.
    if option_type == "call":
        # Black-Scholes formula for a call option.
        # norm.cdf(d1) is the cumulative standard normal distribution function of d1.
        return S * norm.cdf(d1) - K * exp(-r * T) * norm.cdf(d2)
    else:
        # Black-Scholes formula for a put option.
        return K * exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def implied_volatility(price, S, K, T, r, option_type="call"):
    """
    Calculates the implied volatility (IV) of an option using the Black-Scholes model
    and Brent's method to find the root.

    Implied volatility is the volatility that, when input into the Black-Scholes formula,
    yields the current market price of the option.

    Args:
        price (float): The current market price of the option.
        S (float): Current price of the underlying asset.
        K (float): Strike price of the option.
        T (float): Time to expiration in years.
        r (float): Risk-free interest rate (annualized).
        option_type (str): Type of the option, either "call" or "put".

    Returns:
        float or None: The implied volatility, or None if calculation fails (e.g., no real root found).
    """
    try:
        # brentq is a root-finding algorithm. It tries to find a sigma value
        # such that black_scholes_price(S, K, T, r, sigma, option_type) - price = 0.
        # The search range for sigma is typically between a very small positive number (1e-6)
        # and a reasonably large value (5.0, representing 500% volatility).
        return brentq(lambda sigma: black_scholes_price(S, K, T, r, sigma, option_type) - price, 1e-6, 5)
    except ValueError:
        # If brentq cannot find a root within the given interval, a ValueError is raised.
        # This can happen if the market price is inconsistent with the Black-Scholes model
        # for any realistic volatility.
        return None

# ----------------------------
# 2. Get Call Options from FMP
# ----------------------------

def get_call_options_fmp(ticker, api_key):
    """
    Fetches call option chain data for a given ticker from the Financial Modeling Prep (FMP) API.

    Args:
        ticker (str): The stock ticker symbol (e.g., "AAPL").
        api_key (str): Your FMP API key.

    Returns:
        list: A list of dictionaries, where each dictionary represents a call option contract.

    Raises:
        Exception: If the API request fails (e.g., non-200 status code).
    """
    # Construct the API URL for the options chain.
    url = f"https://financialmodelingprep.com/api/v3/options-chain/{ticker}?apikey={api_key}"
    # Send a GET request to the FMP API.
    response = requests.get(url)
    # Check if the HTTP request was successful (status code 200).
    if response.status_code != 200:
        # Raise an exception if the API returns an error.
        raise Exception(f"API Error: {response.status_code}")
    # Parse the JSON response into a Python dictionary/list.
    options_data = response.json()
    # Filter the data to include only call options and return the list.
    return [opt for opt in options_data if opt['contractType'] == 'CALL']

# ----------------------------
# 2. Get Call Options from yfinance
# ----------------------------

def get_call_options_yfinance(ticker):
    """
    Fetches call option chain data for a given ticker using the yfinance library.
    This function aims to replicate the output structure of the FMP API function
    for relevant option details, specifically focusing on call options.

    Args:
        ticker (str): The stock ticker symbol (e.g., "AAPL").

    Returns:
        list: A list of dictionaries, where each dictionary represents a call option contract.
              Each dictionary will contain 'strike', 'lastPrice', 'expirationDate', and 'contractType'.

    Raises:
        Exception: If no option data is found for the given ticker or an error occurs
                   during data retrieval.
    """
    try:
        # Create a Ticker object for the specified stock.
        stock = yf.Ticker(ticker)

        # Get the list of available expiration dates for options.
        # This will be a list of strings in 'YYYY-MM-DD' format.
        expiration_dates = stock.options

        # Initialize an empty list to store all call option data.
        all_call_options = []

        # Iterate through each expiration date to fetch option chains.
        for exp_date_str in expiration_dates:
            try:
                # Get the option chain (calls and puts) for the current expiration date.
                # This returns an object with .calls and .puts DataFrames.
                opt_chain = stock.option_chain(exp_date_str)

                # Access the DataFrame containing call options for this expiration.
                calls_df = opt_chain.calls

                # Check if the calls DataFrame is not empty.
                if not calls_df.empty:
                    # Iterate through each row (option contract) in the calls DataFrame.
                    for index, row in calls_df.iterrows():
                        # Append a dictionary for each call option to our list.
                        # We map yfinance DataFrame columns to the desired output format.
                        all_call_options.append({
                            "strike": row['strike'],
                            # 'lastPrice' is typically the last traded price.
                            # yfinance might have 'lastPrice' or 'lastTradePrice'.
                            # Using 'lastPrice' as per the FMP example.
                            "lastPrice": row['lastPrice'],
                            "expirationDate": exp_date_str, # The current expiration date string.
                            "contractType": "CALL" # Explicitly set as 'CALL'.
                        })
            except Exception as e:
                # Print a warning if an error occurs for a specific expiration date,
                # but continue processing other expirations.
                print(f"Warning: Could not retrieve options for {ticker} on expiration {exp_date_str}. Error: {e}")
                continue # Move to the next expiration date.

        # If no call options were found across all expirations, raise an exception.
        if not all_call_options:
            raise Exception(f"No call option data found for ticker: {ticker}")

        return all_call_options

    except Exception as e:
        # Catch any broader exceptions that might occur during ticker initialization or
        # initial options fetching (e.g., invalid ticker).
        raise Exception(f"Error fetching options for {ticker}: {e}")

    

# ----------------------------
# 3. Calculate IV by Expiration
# ----------------------------

def calculate_iv_curve(ticker, spot_price, risk_free_rate, api_key):
    """
    Calculates the implied volatility for all available call options for a given ticker
    and returns a DataFrame with option details and their calculated IVs.

    Args:
        ticker (str): The stock ticker symbol.
        spot_price (float): The current market price of the underlying stock.
        risk_free_rate (float): The annualized risk-free interest rate (e.g., 0.05).
        api_key (str): Your FMP API key.

    Returns:
        pd.DataFrame: A DataFrame containing 'expirationDate', 'strike', 'lastPrice',
                      and 'impliedVolatility' for each call option, or an empty DataFrame
                      if no valid options are found.
    """
    # Fetch all call options for the specified ticker.
    calls = get_call_options_fmp(ticker, api_key)
    # Initialize an empty list to store dictionaries of IV data.
    iv_data = []

    # Get today's date to calculate time to expiration.
    today = datetime.today()

    # Iterate through each call option fetched from the API.
    for call in calls:
        # Extract relevant option details.
        strike = call['strike']
        last_price = call['lastPrice']
        expiration_str = call['expirationDate'] # Expiration date as a string (e.g., "YYYY-MM-DD").

        try:
            # Convert the expiration date string to a datetime object.
            expiration = datetime.strptime(expiration_str, "%Y-%m-%d")
            # Calculate Time to Expiration (T) in years.
            # (expiration - today).days gives the number of days difference.
            # Divide by 365.0 to convert days into years.
            T = (expiration - today).days / 365.0
            # Skip options that have already expired or have zero time to expiration.
            if T <= 0:
                continue
            # Calculate the implied volatility for the current option.
            iv = implied_volatility(last_price, spot_price, strike, T, risk_free_rate, option_type="call")
            # If a valid implied volatility is returned (not None), append it to the data list.
            if iv:
                iv_data.append({
                    "expirationDate": expiration_str,
                    "strike": strike,
                    "lastPrice": last_price,
                    "impliedVolatility": round(iv, 4) # Round IV to 4 decimal places for readability.
                })
        except Exception as e:
            # Catch any exceptions that might occur during date parsing or IV calculation
            # (e.g., malformed date string, issues with implied_volatility).
            # Print the error for debugging, but continue processing other options.
            # print(f"Error processing option: {call}. Error: {e}") # Uncomment for debugging
            continue # Skip to the next option if an error occurs.

    # Convert the list of IV data dictionaries into a pandas DataFrame.
    return pd.DataFrame(iv_data)

# ----------------------------
# 4. Run Example
# ----------------------------

# This block ensures that the code inside it only runs when the script is executed directly,
# not when it's imported as a module into another script.
if __name__ == "__main__":
    # --- Configuration Parameters ---
    # IMPORTANT: Replace "YOUR_FMP_API_KEY" with your actual Financial Modeling Prep API key.
    api_key = "v0Y7rqjEfz0nBiixKBqJwLLgyFYbOUGA"
    # The stock ticker symbol for which to calculate IV.
    ticker = "AAPL"
    # The current spot price of the underlying stock.
    # In a real-world application, this would be fetched dynamically from a live market data source.
    spot_price = 190.0
    # The risk-free interest rate, expressed as a decimal (e.g., 0.05 for 5%).
    # This is typically approximated using the yield of a short-term government bond.
    risk_free_rate = 0.05

    # Call the main function to calculate the implied volatility curve.
    iv_df = calculate_iv_curve(ticker, spot_price, risk_free_rate, api_key)

    # --- Data Aggregation and Display ---
    # Group the DataFrame by 'expirationDate' and calculate the mean (average)
    # implied volatility for each expiration.
    grouped = iv_df.groupby("expirationDate")["impliedVolatility"].mean().reset_index()
    # Rename the columns for clarity: 'date' for expiration date and 'avgImpliedVolatility'.
    grouped.columns = ["date", "avgImpliedVolatility"]

    # Print the resulting DataFrame, showing the average implied volatility for each expiration date.
    print(grouped)

Exception: Error fetching options for AAPL: Too Many Requests. Rate limited. Try after a while.